# Usar LLMs para generar resúmenes de propiedades de `Zonaprop`
- https://www.machinelearnear.com/
- https://www.youtube.com/@machinelearnear
- Webinar real estate: https://www.youtube.com/watch?v=DuujwXarVbY&ab_channel=SantiagoMagnin

In [1]:
import os
import pandas as pd

def abrir_zonaprop_csv(fname, processed_folder = './processed'):
    # listamos las subcarpetas en la carpeta
    subfolders = [f.path for f in os.scandir(processed_folder) if f.is_dir()]
    
    # si no hay subcarpetas, retornamos
    if not subfolders:
        print(f"No hay subcarpetas en {processed_folder}.")
        return None

    # imprimimos las subcarpetas para que el usuario elija una
    print("Subcarpetas disponibles:")
    for idx, folder in enumerate(subfolders):
        print(f"{idx + 1}. {folder}")

    # pedimos al usuario que elija una subcarpeta
    choice = int(input("\nSeleccioná el número de la subcarpeta: "))
    selected_folder = subfolders[choice - 1]

    # construimos la ruta al archivo csv
    csv_file = os.path.join(selected_folder, fname)

    # verificamos si el archivo existe
    if not os.path.exists(csv_file):
        print(f"No se encontró el archivo '{csv_file}'.")
        return None

    # leemos y retornamos el dataframe
    return pd.read_csv(csv_file)

In [3]:
# load data from local
zonaprop_listings = abrir_zonaprop_csv('zonaprop_with_userviews.csv')
zonaprop_listings.head()

Subcarpetas disponibles:
1. ./processed/inmuebles-venta-palermo-colegiales-villa-crespo-chacarita-con-balcon-y-disposicion-frente-mas-30-m2-cubiertos-50000-130000-dolar
2. ./processed/.ipynb_checkpoints
3. ./processed/inmuebles-venta-barrio-norte-palermo-colegiales-villa-crespo-publicado-hace-menos-de-45-dias-50000-130000-dolar-orden-visitas-descendente



Seleccioná el número de la subcarpeta:  3


,listing_url,asking_price_in_usd,expensas_in_ars,latitude,longitude,google_maps,photos,whatsapp,published_on,disposición,...,cantidaddedepartamentos,saladejuegos,promotiondescription,tipoderubro,cantidaddeascensores,cantidaddelocalescomerciales,user_views,days,views_per_day,usd_per_m2
0,https://www.zonaprop.com.ar/propiedades/clasif...,98000,30000.0,-34.5834184,-58.404087,"https://maps.google.com/?q=-34.5834184,-58.404087",['https://imgar.zonapropcdn.com/avisos/1/00/52...,5.491166e+12,24-01-2024,Frente,...,NaN,NaN,NaN,NaN,NaN,NaN,169,15,11,2450.0
1,https://www.zonaprop.com.ar/propiedades/clasif...,114000,1.0,-34.5917978,-58.4234218,"https://maps.google.com/?q=-34.5917978,-58.423...",['https://imgar.zonapropcdn.com/avisos/resize/...,NaN,18-01-2024,Frente,...,NaN,NaN,NaN,NaN,NaN,NaN,170,27,6,2111.0
2,https://www.zonaprop.com.ar/propiedades/clasif...,99000,19500.0,-34.57885220000001,-58.4136051,"https://maps.google.com/?q=-34.57885220000001,...",['https://imgar.zonapropcdn.com/avisos/1/00/53...,NaN,05-02-2024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,170,8,21,3300.0
3,https://www.zonaprop.com.ar/propiedades/clasif...,51000,NaN,-34.5987945,-58.38669719999996,"https://maps.google.com/?q=-34.5987945,-58.386...",['https://imgar.zonapropcdn.com/avisos/1/00/52...,5.491146e+12,09-01-2024,Lateral,...,NaN,NaN,NaN,NaN,NaN,NaN,170,42,5,1758.0
4,https://www.zonaprop.com.ar/propiedades/clasif...,75000,NaN,-34.5920267,-58.4334159,"https://maps.google.com/?q=-34.5920267,-58.433...",['https://imgar.zonapropcdn.com/avisos/1/00/52...,NaN,02-01-2024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,171,37,5,2205.0


## Como traemos la información de `Airbnb`?

In [4]:
from sklearn.neighbors import BallTree

def find_within_radius(reference_df, target_df, radius_km=1):
    # Convert radius from kilometers to radians for BallTree
    radius_rad = radius_km / 6371  # Earth's radius in km

    # Creating BallTree with target data
    tree = BallTree(np.deg2rad(target_df[['latitude', 'longitude']].values), metric='haversine')

    # Initialize DataFrame to store results
    results_df = pd.DataFrame()

    # Iterate over reference DataFrame
    for index, row in reference_df.iterrows():
        # Query BallTree
        indices = tree.query_radius(np.deg2rad([[row['latitude'], row['longitude']]]), r=radius_rad)
        # Extract relevant rows from target DataFrame
        filtered_df = target_df.iloc[indices[0]]
        results_df = pd.concat([results_df, filtered_df])

    return results_df

In [5]:
airbnb_listings = pd.read_csv('processed/airbnb_listings.csv')
# airbnb_reviews = pd.read_csv('processed/airbnb_reviews.csv')

### Encontrar los listings mas cercanos

In [6]:
def find_within_radius(row, airbnb_listings, radius_km):
    radius_rad = radius_km / 6371
    valid_listings = airbnb_listings.dropna(subset=['latitude', 'longitude'])
    tree = BallTree(np.deg2rad(valid_listings[['latitude', 'longitude']].values), metric='haversine')
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        indices = tree.query_radius(np.deg2rad([[row['latitude'], row['longitude']]]), r=radius_rad)
        return valid_listings.iloc[indices[0]]
    return pd.DataFrame()

In [66]:
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd

def agregar_info_airbnb(listings, airbnb_listings, radius_km):
    listings = listings.copy()
    listings['latitude'] = pd.to_numeric(listings['latitude'], errors='coerce')
    listings['longitude'] = pd.to_numeric(listings['longitude'], errors='coerce')

    def find_within_radius(row, radius_km):
        radius_rad = radius_km / 6371
        valid_listings = airbnb_listings.dropna(subset=['latitude', 'longitude'])
        tree = BallTree(np.deg2rad(valid_listings[['latitude', 'longitude']].values), metric='haversine')
        if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
            indices = tree.query_radius(np.deg2rad([[row['latitude'], row['longitude']]]), r=radius_rad)
            return valid_listings.iloc[indices[0]]
        return pd.DataFrame()

    for index, row in listings.iterrows():
        closest_listings = find_within_radius(row, radius_km)
        if closest_listings.empty: continue
        filtered = closest_listings.dropna(subset=['estimated_price_per_night_in_USD', 'review_scores_rating', 
                           'review_scores_location', 'review_scores_value', 'room_type', 'estimated_nights_booked_l30d'])

        # calcular la probabilidad de alquiler
        booking_counts = filtered['estimated_nights_booked_l30d'].value_counts()
        probabilidad_alquiler = 'más probable' if (booking_counts.get('high', 0) > booking_counts.get('low', 0)
                                                  ) or (booking_counts.get('high', 0) > booking_counts.get('medium', 0)) else 'menos probable'
        listings.at[index, 'airbnb_probabilidad_alquiler'] = probabilidad_alquiler
        
        # otros numeros grosos
        listings.at[index, 'airbnb_avg_price_entire_home'] = int(filtered[filtered['room_type'] == 'Entire home/apt']['estimated_price_per_night_in_USD'].mean())
        listings.at[index, 'airbnb_avg_price_private_room'] = int(filtered[filtered['room_type'] == 'Private room']['estimated_price_per_night_in_USD'].mean())
        listings.at[index, 'airbnb_avg_review_score_rating'] = round(filtered['review_scores_rating'].mean(),2)
        listings.at[index, 'airbnb_avg_review_score_location'] = round(filtered['review_scores_location'].mean(),2)
        listings.at[index, 'airbnb_avg_review_score_value'] = round(filtered['review_scores_value'].mean(),2)

    return listings

#### Mostrame un ejemplo che

In [67]:
top_listings = zonaprop_listings[zonaprop_listings.views_per_day > 60].reset_index(drop=True)
top_listings = high_relevance_listings.dropna(axis=1, how='all')

In [68]:
# suponiendo que `top_listings` y `airbnb_listings` son tus DataFrames
a = agregar_info_airbnb(top_listings, airbnb_listings, 0.3)

In [69]:
a.head()

,listing_url,asking_price_in_usd,expensas_in_ars,latitude,longitude,google_maps,photos,whatsapp,published_on,disposición,...,user_views,days,views_per_day,usd_per_m2,airbnb_probabilidad_alquiler,airbnb_avg_price_entire_home,airbnb_avg_price_private_room,airbnb_avg_review_score_rating,airbnb_avg_review_score_location,airbnb_avg_review_score_value
0,https://www.zonaprop.com.ar/propiedades/clasif...,95000,57000.0,-34.588879,-58.413855,"https://maps.google.com/?q=-34.5888791,-58.413...",['https://imgar.zonapropcdn.com/avisos/1/00/53...,NaN,08-02-2024,Contrafrente,...,66,1,66,1938.0,menos probable,59.0,27.0,4.78,4.94,4.73
1,https://www.zonaprop.com.ar/propiedades/clasif...,81650,5000.0,-34.582257,-58.453389,"https://maps.google.com/?q=-34.5822573,-58.453...",['https://imgar.zonapropcdn.com/avisos/1/00/52...,5.491127e+12,01-02-2024,NaN,...,1402,22,63,1600.0,más probable,49.0,18.0,4.82,4.77,4.73
2,https://www.zonaprop.com.ar/propiedades/clasif...,75000,41000.0,-34.589921,-58.423228,"https://maps.google.com/?q=-34.5899212,-58.423...",['https://imgar.zonapropcdn.com/avisos/resize/...,5.491150e+12,08-02-2024,Frente,...,948,10,94,2083.0,menos probable,66.0,23.0,4.79,4.93,4.73
3,https://www.zonaprop.com.ar/propiedades/clasif...,130000,0.0,-34.573193,-58.440290,"https://maps.google.com/?q=-34.5731928,-58.440...",['https://imgar.zonapropcdn.com/avisos/1/00/52...,5.491166e+12,08-02-2024,Frente,...,610,10,61,1969.0,más probable,52.0,26.0,4.73,4.82,4.67


## Generar un resumen con LLM

### Primero, configuramos todo
- https://github.com/BerriAI/litellm/
- https://docs.litellm.ai/docs/providers/togetherai
- listado de cuanto salen las API de Mixtral-8x7B: https://twitter.com/chiefaioffice/status/1750622940658299332

In [70]:
# !pip install litellm

In [71]:
from dotenv import load_dotenv   #for python-dotenv method
load_dotenv()                    #for python-dotenv method
import os 

In [87]:
system_instructions = """Always follow these instructions:
- Using the above context only, return a single paragraph summarizing all the information as your output.
- Write with strong Argentinian accent in Spanish.
- Be descriptive, don't skip words.
- For "Zonaprop", pay special attention to: 'asking_price_in_usd', 'expensas_in_ars', 'views_per_day', 'usd_per_m2'
- For "Airbnb", pay special attention to: 'average price', 'review score location', and `probabilidad de alquiler'. All prices are in USD.
- Be casual unless otherwise specified
- Be accurate, thorough, and descriptive
- No need to disclose you're an AI, nor greet me, or say goodbye.
"""

In [73]:
def info_del_listing(row):
    zonaprop = '\n'.join(f"- {col}: {row[col]}" for col in row.index if col not in 
                         ['photos', 'listing_url', 'latitude', 'longitude', 'google_maps', 'whatsapp', 'days'] and
                         'airbnb_' not in col and pd.notnull(row[col]))
    airbnb = '\n'.join(f"- {col}: {row[col]}" for col in row.index if 'airbnb_' in col)
    return zonaprop, airbnb

In [74]:
b = info_del_listing(a.iloc[1])
prompt = f"Return a dense summary from the given context:\n```Zonaprop:{b[0]}\n----\nAirbnb listings nearby:\n{b[1]}```"

In [75]:
from litellm import completion 

messages = [
    {"role": "system", "content": system_instructions},
    {"role": "user", "content": prompt},
]

### Después, probamos distintos modelos a ver cual nos gusta mas

`together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1` @ $0.60/M

In [76]:
chat_completion = completion(
    messages=messages,
    model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
    max_tokens=1024
)

In [77]:
print(chat_completion.choices[0].message.content)

La lista de Zonaprop tiene un precio de lista en dólares de 81650, con expensas en pesos argentinos de 5000 y publicada el 01-02-2024. Tiene un tamaño total y cubierto de 51 metros cuadrados, con 3 ambientes, 1 baño, una antigüedad de 5 años, 1 planta, 9 metros cuadrados de superficie semicubierta, 2 dormitorios, y sin frente ni largo de terreno. Ha recibido 1402 visitas de usuarios y 63 visitas por día, con un precio de dólares por metro cuadrado de 1600.

En cuanto a los alquileres de Airbnb en las cercanías, la probabilidad de alquiler es más probable, con un precio promedio de 49 dólares para un hogar entero y 18 dólares para una habitación privada. La puntuación promedio de reseñas es de 4.82 para la calidad general, 4.77 para la ubicación y 4.73 para el valor.


`NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO` @ $0.60/M

In [78]:
chat_completion = completion(
    messages=messages,
    model="together_ai/NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO",
    max_tokens=1024
)

In [79]:
print(chat_completion.choices[0].message.content)

La propiedad en Zonaprop tiene un precio de venta de 81,650 USD, gastos de expensas de 50,00 ARS, y se publicó el 1 de febrero de 2024. Tiene una superficie total y cubierta de 51 m², con 3 ambientes, 1 baño, una antigüedad de 5 años, 1 planta, y una superficie semicubierta de 9 m². Cuenta con 2 dormitorios, y un terreno de 0 m² de frente y largo. La propiedad ha recibido 1,402 visitas y tiene un promedio de 63 visitas por día, con un valor de 1,600 USD por m² de superficie cubierta.
En cuanto a los alquileres en Airbnb cercanos, es más probable que se alquilen propiedades enteras por un precio promedio de 49 USD, mientras que las habitaciones privadas tienen un precio promedio de 18 USD. Los anfitriones tienen una puntuación promedio de 4.82, con una puntuación promedio de 4.77 para la ubicación y de 4.73 para el valor.


`teknium/OpenHermes-2p5-Mistral-7B` @ $0.20/M

In [80]:
chat_completion = completion(
    messages=messages,
    model="together_ai/teknium/OpenHermes-2p5-Mistral-7B",
    max_tokens=1024
)

In [81]:
print(chat_completion.choices[0].message.content)

Hola amigo, te voy a contar sobre una propiedad que encontré en Zonaprop. La casa tiene un precio de venta de 81.650 dólares, pero ten en cuenta que hay gastos adicionales de 5.000 pesos argentinos. La casa fue publicada el 1 de febrero de 2024 y tiene un tamaño total de 51 metros cuadrados, con 51 metros cuadrados cubiertos. Tiene 3 ambientes, 1 baño, y es de 5 años de antigüedad. La casa tiene 1 piso y 9 metros cuadrados y medio cubiertos. También tiene 2 dormitorios, 0 metros de frente de terreno y 0 metros de largo de terreno. La casa ha sido vista 1.402 veces y es vista en promedio 63 veces al día. El precio es de 1.600 dólares por metro cuadrado.

Ahora, cerca de esa casa, hay algunas casas en Airbnb. La probabilidad de alquilar es más probable, y el precio promedio por casa entera es de 49 dólares. El precio promedio por habitación privada es de 18 dólares. Las calificaciones promedio para la ubicación, la calidad y el valor son de 4.77, 4.82 y 4.73, respectivamente.

Espero que

`Qwen/Qwen1.5-14B-Chat` @ $0.30/M

In [105]:
chat_completion = completion(
    messages=messages,
    model="together_ai/Qwen/Qwen1.5-14B-Chat",
    max_tokens=1024
)

In [106]:
print(chat_completion.choices[0].message.content)


En Zonaprop, hay una propiedad en venta con un precio de venta de $81,650 dólares estadounidenses, incluyendo gastos de $5,000 pesos argentinos mensuales. La propiedad cuenta con 51 metros cuadrados de superficie total y cubierta, 3 habitaciones, un baño y una antigüedad de 5 años. Está ubicada en una sola planta y tiene una superficie semi-cubierta de 9 metros cuadrados. Ha recibido 1,402 visitas y promedio de 63 vistas diarias, con un costo de $1,600 dólares por metro cuadrado. Por otro lado, en las listas de Airbnb cercanas, el alquiler es considerado más probable, con precios promedio de $49 por el alojamiento entero y $18 por una habitación privada. Las calificaciones promedio son buenas, con puntajes de 4.82 en valor, 4.77 en ubicación y 4.73 en la calidad del servicio.



### Lo formateamos en `Markdown` para que quede mas lindo

In [137]:
def llm_response(row, model="together_ai/NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"):
    info = info_del_listing(row)
    prompt = f"Return a dense summary from the given context:\n```Zonaprop:{info[0]}\n----\nAirbnb listings nearby:\n{info[1]}```"
    messages = [
        {"role": "system", "content": system_instructions},
        {"role": "user", "content": prompt},
    ]
    chat_completion = completion(
        messages=messages,
        model=model,
        max_tokens=1024
    )
    return chat_completion.choices[0].message.content

In [138]:
import IPython.display as display
from IPython.display import Markdown, HTML

def mostrar_resumen_propiedad(row):
    # nos traemos la respuesta del llm
    llm_summary = llm_response(row)
    
    # formateamos el nro de wa
    if pd.isna(row['whatsapp']):
        whatsapp_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
    else:
        whatsapp_url = f"https://wa.me/{int(row['whatsapp'])}?text=Que%20haces%20che,%20te%20quiero%20comprar%20esto,%20abrazo:%20{row['listing_url']}"
 
    # traemos el resto de la info en markdown
    caracteristicas = '\n'.join(f"- {col}: {row[col]}" for col in row.index if col not in ['photos', 'listing_url'] and 'airbnb_' not in col)

    markdown_resumen = f"""
---
**[#{row.name}] {row['listing_url']}**

- **Mapa de donde está:** {row['google_maps']}
- **Mandale un whatsapp:** [apretá acá que se abre solo]({whatsapp_url})
- **Resumen:** {llm_summary}
"""

    # mostrar el resumen en markdown
    display.display(Markdown(markdown_resumen))

    # agregar fotos en html con grilla
    if row['photos']:
        fotos = row['photos'][1:-1].replace("'", "").split(', ')
        fotos_html = '<div style="display: grid; grid-template-columns: repeat(3, 1fr); grid-gap: 5px;">'
        fotos_html += ''.join(f'<div style="width: 100%;"><img src="{url.strip()}" style="width: 100%; max-height: 200px; object-fit: cover;" /></div>' for url in fotos[:6])
        fotos_html += '</div>'
        display.display(HTML(fotos_html))

In [139]:
# uso de la función para la primera fila
mostrar_resumen_propiedad(a.loc[1])


---
**[#1] https://www.zonaprop.com.ar/propiedades/clasificado/veclphin-venta-depto-tipo-ph-3-amb-baulera-colegiales-52938413.html**

- **Mapa de donde está:** https://maps.google.com/?q=-34.5822573,-58.4533889
- **Mandale un whatsapp:** [apretá acá que se abre solo](https://wa.me/5491126885078?text=Que%20haces%20che,%20te%20quiero%20comprar%20esto,%20abrazo:%20https://www.zonaprop.com.ar/propiedades/clasificado/veclphin-venta-depto-tipo-ph-3-amb-baulera-colegiales-52938413.html)
- **Resumen:** En Zonaprop, se encuentra una propiedad en venta con un precio de 81,650 dólares estadounidenses, gastos comunes de 50,00 pesos argentinos, publicada el 1 de febrero de 2024. La propiedad cuenta con una superficie total y cubierta de 51 metros cuadrados, 3 ambientes, 1 baño, una antigüedad de 5 años, 1 planta, una superficie semicubierta de 9 metros cuadrados, 2 dormitorios, y un terreno de 0 metros cuadrados de frente y largo. La propiedad ha recibido 1,402 visitas y actualmente tiene un promedio de 63 visitas por día. El precio por metro cuadrado es de 1,600 dólares estadounidenses.
En cuanto a los alquileres en Airbnb cercanos, es más probable que se alquilen propiedades enteras a un precio promedio de 49 dólares estadounidenses por noche, mientras que las habitaciones privadas tienen un precio promedio de 18 dólares por noche. La propiedad en Airbnb tiene una puntuación promedio de 4.82 en la calificación general, 4.77 en la ubicación y 4.73 en el valor.


## Ahora sí, definimos un criterio para filtrar las propiedades

In [140]:
from tqdm import tqdm

In [141]:
def hacer_analisis(df, min_views=60, max_views=float('inf'), airbnb_listings=airbnb_listings, radius_km=0.3, show_top=10):
    listings = df[(df.views_per_day > min_views) & (df.views_per_day <= max_views)].reset_index(drop=True)
    listings = listings.head(show_top)
    # listings = listings.dropna(axis=1, how='all')
    listings = agregar_info_airbnb(listings, airbnb_listings, radius_km)
    for index, row in tqdm(listings.iterrows(), desc="Llamando APIs .."):
        mostrar_resumen_propiedad(row)

Nos vamos a basar en esto que dice el groso de [Santiago Magnin](https://twitter.com/santivende) en este video [*"GUÍA 2024 para COMPRADORES 💣 Webinar bomba 💣 ¿Cómo comprar con Javier Milei Presidente?"*](https://www.youtube.com/watch?v=DuujwXarVbY&ab_channel=SantiagoMagnin).

En resumen, para `Zonaprop`:
- (1) ~60 visitas/dia, se reservan en la primera semana, negociación promedio del 2%
- (2) ~30 visitas/dia, se reservan entre 8 y 30 dias, negociación promedio del 5%
- (3) ~20 visitas/dia, se reservan entre 31 y 60 dias, negociación promedio del 7%
- (4) ~10 visitas/dia, se reservan en más de 60 dias, negociación promedio del 8%

In [142]:
hacer_analisis(zonaprop_listings, show_top=10)

Llamando APIs ..: 0it [00:00, ?it/s]


---
**[#0] https://www.zonaprop.com.ar/propiedades/clasificado/veclapin-vidt-y-guemes-oportunidad!-49-m-sup2--a-nuevo!-a-1-53058867.html**

- **Mapa de donde está:** https://maps.google.com/?q=-34.5888791,-58.4138553
- **Mandale un whatsapp:** [apretá acá que se abre solo](https://www.youtube.com/watch?v=dQw4w9WgXcQ)
- **Resumen:** En Zonaprop, se encuentra una propiedad en venta con un precio de 95,000 USD, gastos comunes de 57,00 ARS, publicada el 08-02-2024, ubicada en contrafrente, con una superficie total y cubierta de 49 m², 2 ambientes, 1 baño, 50 años de antigüedad, 1 plantas, y sin superficie semicubierta. La propiedad ha recibido 66 visitas de usuarios y tiene un promedio de 66 visitas por día, con un valor de 1938 USD por m² cubierto.
En cuanto a los alquileres en Airbnb cercanos, la probabilidad de alquiler es menos probable. El precio promedio de una casa entera es de 59 USD, mientras que el precio promedio de una habitación privada es de 27 USD. La propiedad tiene una puntuación promedio de 4.78 en valoración general, 4.94 en ubicación y 4.73 en valoración del huésped.


Llamando APIs ..: 1it [00:07,  7.15s/it]


---
**[#1] https://www.zonaprop.com.ar/propiedades/clasificado/veclphin-venta-depto-tipo-ph-3-amb-baulera-colegiales-52938413.html**

- **Mapa de donde está:** https://maps.google.com/?q=-34.5822573,-58.4533889
- **Mandale un whatsapp:** [apretá acá que se abre solo](https://wa.me/5491126885078?text=Que%20haces%20che,%20te%20quiero%20comprar%20esto,%20abrazo:%20https://www.zonaprop.com.ar/propiedades/clasificado/veclphin-venta-depto-tipo-ph-3-amb-baulera-colegiales-52938413.html)
- **Resumen:** En Zonaprop, se encuentra una propiedad en venta con un precio de 81,650 dólares estadounidenses, gastos comunes de 50,00 pesos argentinos, publicada el 1 de febrero de 2024. La propiedad cuenta con una superficie total y cubierta de 51 metros cuadrados, 3 ambientes, 1 baño, una antigüedad de 5 años, 1 planta, una superficie semicubierta de 9 metros cuadrados, 2 dormitorios, y un terreno de 0 metros cuadrados de frente y largo. La propiedad ha recibido 1,402 visitas y actualmente tiene un promedio de 63 visitas por día. El precio por metro cuadrado es de 1,600 dólares estadounidenses.
En cuanto a los alquileres en Airbnb cercanos, es más probable que se alquilen propiedades enteras a un precio promedio de 49 dólares estadounidenses por noche, mientras que las habitaciones privadas tienen un precio promedio de 18 dólares por noche. La propiedad en Airbnb tiene una puntuación promedio de 4.82 en la calificación general, 4.77 en la ubicación y 4.73 en el valor.


Llamando APIs ..: 2it [00:15,  7.92s/it]


---
**[#2] https://www.zonaprop.com.ar/propiedades/clasificado/veclapin-departamento-en-duplex-a-refaccionar-excelente-52996658.html**

- **Mapa de donde está:** https://maps.google.com/?q=-34.5899212,-58.4232282
- **Mandale un whatsapp:** [apretá acá que se abre solo](https://wa.me/5491149939812?text=Que%20haces%20che,%20te%20quiero%20comprar%20esto,%20abrazo:%20https://www.zonaprop.com.ar/propiedades/clasificado/veclapin-departamento-en-duplex-a-refaccionar-excelente-52996658.html)
- **Resumen:** En Zonaprop, se encuentra un departamento en venta con un precio de 75,000 USD, gastos comunes de 41,00 ARS, publicado el 08-02-2024, ubicado en frente y muy luminoso. Tiene una superficie total de 36 m², de los cuales 34 m² son superficie cubierta. Cuenta con 2 ambientes, 1 baño, 5 departamentos por piso y 6 pisos en el edificio. El departamento tiene 50 años de antigüedad, es apto para crédito y tiene 2 plantas. Ha recibido 948 visitas de usuarios y 94 vistas por día. El precio por metro cuadrado es de 2,083.00 USD.
En cuanto a Airbnb, en la zona hay alquileres de propiedades completas con un precio promedio de 66 USD, alquileres de habitaciones privadas con un precio promedio de 23 USD, y una puntuación promedio de 4.79 en la calidad general, 4.93 en la ubicación y 4.73 en el valor. La probabilidad de alquiler en la zona es menos probable.


Llamando APIs ..: 3it [00:22,  7.64s/it]


---
**[#3] https://www.zonaprop.com.ar/propiedades/clasificado/veclphin-ph-en-colegiales-52997811.html**

- **Mapa de donde está:** https://maps.google.com/?q=-34.5731928,-58.4402904
- **Mandale un whatsapp:** [apretá acá que se abre solo](https://wa.me/5491165517219?text=Que%20haces%20che,%20te%20quiero%20comprar%20esto,%20abrazo:%20https://www.zonaprop.com.ar/propiedades/clasificado/veclphin-ph-en-colegiales-52997811.html)
- **Resumen:** En Zonaprop, se encuentra una propiedad en venta con un precio de 130,00 USD, sin gastos comunes en pesos argentinos. Publicada el 08-02-2024, esta propiedad cuenta con una superficie total y cubierta de 66 m², con 3 ambientes, 1 pisos, 1 baños y 2 dormitorios. La antigüedad es de 70 años y no se proporciona información sobre el terreno. La propiedad ha recibido 610 visitas y 61 visitas por día, con un valor de 1969 USD por m² cubierto.
En cuanto a Airbnb, se encuentran alquileres cercanos con una mayor probabilidad de alquiler. El precio promedio para una casa entera es de 52 USD, mientras que para una habitación privada es de 26 USD. La propiedad promedio tiene una puntuación de 4.73 en valoración general, 4.82 en ubicación y 4.67 en valoración del alojamiento.


Llamando APIs ..: 4it [00:28,  7.06s/it]
